In [ ]:
import numpy as np, torch, time
from Ctubes.curve_parametrizations import trefoil_parameterization
from Ctubes.geometry_utils import (
    rotate_about_axis, regular_polygon, 
    compute_binormals, get_bisecting_plane_normals_with_symmetry
)
from Ctubes.plot_utils import plot_generatrix
from Ctubes.tubes import Directrix, Generatrix, CTube
from Ctubes.opt import CTubeOptimizationProblem
from Ctubes.misc_utils import get_pairings_exact
from Ctubes.path_utils import get_name, setup_paths

paths = setup_paths(get_name())

PI = np.pi
TORCH_DTYPE = torch.float64
torch.set_default_dtype(TORCH_DTYPE)
torch.set_printoptions(precision=4)

%load_ext autoreload
%autoreload 2

%matplotlib widget

# Figure 19: Fodable Accordion Trefoil

In [ ]:
# Generatrix: regular N-gon
N = 4
tube_radius = 0.3
generatrix_2d = regular_polygon(N, tube_radius)

fig, ax = plot_generatrix(generatrix_2d)

In [ ]:
# DIrectrix: coarse trefoil knot with binormal offsets 
closed_curve = True
K_per_lobe = 12
n_fold_symm = 3
K = K_per_lobe * n_fold_symm + 1
M = K

# Define the spline
ts_disc_curve = torch.linspace(0.0, 2.0 * PI, K)
cps_ref = trefoil_parameterization(ts_disc_curve)

# z-scale
cps_ref[:, 2] *= 1.5

# Offset points along +/- the curve binormal
binormals = compute_binormals(cps_ref)
offset_dir = binormals
offset_dir[1::2] = -offset_dir[1::2]
offset_dir *= 0.8 * tube_radius
cps_ref[1:K_per_lobe] += offset_dir[1:K_per_lobe]
cps_ref[-1] = cps_ref[0]

In [ ]:
# Define symmetry transforms
z_axis = torch.tensor([0.0, 0.0, 1.0], dtype=TORCH_DTYPE)
symm_3_fold = [
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(4*PI/3)),
    lambda pts: rotate_about_axis(pts, z_axis, torch.tensor(8*PI/3)),
]
symmetry_transforms = symm_3_fold

# Select one third of the curve
K = (K - 1) // n_fold_symm + 1
M = (M - 1) // n_fold_symm + 1
cps_ref = cps_ref[:K]

In [ ]:
# C-tube
directrix = Directrix(cps_ref, M, symmetry_transforms=symmetry_transforms)

X0 = directrix.X[0]
T0 = directrix.get_tangents()[0]
generatrix = Generatrix(generatrix_2d, X0, T0)


plane_normals = get_bisecting_plane_normals_with_symmetry(directrix)

tube = CTube(
    directrix, generatrix, plane_normals,
    symmetry_transforms=symmetry_transforms
)

In [ ]:
fig, ax = tube.plot_3d()

In [ ]:
fig, ax = tube.plot_unrolled_strips()

### Optimization

In [ ]:
# Set up optimization problem

opt_weights = {
    'preserve_curve': 1e-2 / directrix.aabb_diagonal_length() ** 2,
    'join_ends': 1.0 / generatrix.aabb_diagonal_length() ** 2,
}

objective_args = {
    'join_ends_pairings': get_pairings_exact(N),
}

opt_prob = CTubeOptimizationProblem(
    tube, 
    opt_weights, 
    objective_args,
)

opt_prob.activate_cps(True)
opt_prob.activate_theta(False)
opt_prob.activate_apex_loc_func(False)
opt_prob.activate_plane_normals(False)

In [ ]:
opt_prob.opt_weights['smooth_plane_normal_diffs'] = 0.0  # deactivate plane normal smoothing

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
from scipy.optimize import minimize
from Ctubes.opt import obj_and_grad

torch.autograd.set_detect_anomaly(False)

# Set up optimization configuration
opt_prob.configure_optimization_output(paths)

# Get initial parameters
params0 = opt_prob.get_params_numpy()

# Define objective and gradient function for SciPy
obj_and_grad_scipy = lambda params: obj_and_grad(params, opt_prob)

# Fix variables via double-sided bounds
fixed_indices = []
dofs = opt_prob.get_dof_indices_cps()
fixed_indices += dofs[:3] + dofs[-3:]  # fix endpoints
fixed_indices = list(set(fixed_indices))  # remove duplicates
fixed_indices = sorted(fixed_indices)

bounds = [(None, None)] * len(params0)
for idx in fixed_indices:
    bounds[idx] = (params0[idx], params0[idx])
print(f"Fixing {len(fixed_indices)} parameters.")

# Add initial state to history
opt_prob.add_objective_to_history()

In [ ]:
# Run optimization
start_time = time.time()
result = minimize(
    obj_and_grad_scipy, 
    params0, 
    jac=True, 
    method='L-BFGS-B',
    options={'ftol': 1.0e-10, 'gtol': 1.0e-5, 'disp': True, 'maxiter': 2000},
    bounds=bounds,
    callback=opt_prob.optimization_callback,
)
result.execution_time = time.time() - start_time

# Finalize optimization (save results, render videos, cleanup)
opt_prob.finalize_optimization(result)

In [ ]:
opt_prob.compute_objective(print_to_console=True)

In [ ]:
fig, ax = opt_prob.plot_objective_history()

In [ ]:
fig, ax = opt_prob.plot_3d()

In [ ]:
fig, ax = opt_prob.plot_unrolled_strips()